This notebook can be rerun as often as needed

In [52]:
%load_ext kedro.ipython
%reload_kedro
%load_ext autoreload
%autoreload 2
%config IPCompleter.use_jedi=False

import os
catalog = context.catalog
params = context.params
credentials = context._get_config_credentials()
os.chdir(str(context.project_path))

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


[06/26/23 20:32:00] INFO     Resolved project path as:                                              ]8;id=861137;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=385590;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#136\136]8;;\
                             /Users/ejwillemse/dev/projects/project-rdi-cape-town.                                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/26/23 20:32:01] INFO     Kedro project Project RDI Cape Town                                    ]8;id=448043;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=172968;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#105\105]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=820648;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=94889;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#106\106]8;;\
                             'pipelines'                                                                           

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
from keplergl import KeplerGl

In [137]:
import logging
from typing import Tuple
import pandas as pd
from datetime import datetime
import geopandas as gpd
import osmnx as ox

import logging

logger = logging.getLogger(__name__)


import pandas as pd
import geopandas as gpd


def convert_to_geopandas(
    df: pd.DataFrame, lat_col: str = "lat", lon_col: str = "lon"
) -> gpd.GeoDataFrame:
    """
    Converts a pandas DataFrame with latitude and longitude columns into a GeoDataFrame.

    Parameters:
    df (pandas.DataFrame): The input DataFrame containing the latitude and longitude data.
    lat_col (str): The name of the column containing the latitude values. Default is "lat".
    lon_col (str): The name of the column containing the longitude values. Default is "lon".

    Returns:
    gpd.GeoDataFrame: A GeoDataFrame containing the input data with a "geometry" column of Point objects.
    """
    # Create a GeoDataFrame from the input DataFrame and the latitude and longitude columns
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df[lon_col], df[lat_col]), crs="EPSG:4326"
    )

    return gdf


def get_convex_hull(gdf: gpd.clip, buffer_size=50) -> gpd.GeoDataFrame:
    """
    Calculates the convex hull of a set of points and adds a buffer around it.

    Parameters:
    gpd (gpd.GeoDataFrame): The input DataFrame containing the point data.
    buffer_size (float): The size of the buffer to add around the convex hull, in meters. Default is 50.

    Returns:
    gpd.GeoDataFrame: A GeoDataFrame containing the convex hull of the points with a buffer around it.
    """

    # Log the size of the input data frame
    num_points = len(gdf)
    logger.info(f"Input data frame contains {num_points} points.")

    logger.info("Calculating the convex hull of the points...")
    # Calculate the convex hull of the points
    convex_hull = gdf.geometry.unary_union.convex_hull

    logger.info("Converting the buffer to a GeoSeries...")
    # Convert the buffer to a GeoSeries
    convex_hull_series = gpd.GeoSeries(convex_hull, crs=gdf.crs)

    logger.info("Converting the buffer to the target projection...")
    # Convert the buffer to the target projection
    target_proj = "epsg:3857"
    convex_hull_target = convex_hull_series.to_crs(target_proj)

    # Log the area of the convex hull
    area = convex_hull_target.area.values[0] / (1000 * 1000)
    logger.info(f"Convex hull area is {area:.2f} square km.")

    logger.info(f"Adding a {buffer_size}m buffer around the convex hull...")
    # Add a buffer around the convex hull
    convex_hull_buffer = convex_hull_target.buffer(buffer_size)
    area = convex_hull_buffer.area.values[0] / (1000 * 1000)
    logger.info(f"Convex hull area is now {area:.2f} square km.")

    logger.info("Converting the buffer back to the original projection...")
    # Convert the buffer back to the original projection
    convex_hull_buffer_4326 = convex_hull_buffer.to_crs(gdf.crs)

    return convex_hull_buffer_4326


def get_network(
    gdf: gpd.GeoDataFrame, lat_col: str = "lat", lon_col: str = "lon"
) -> Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame]:
    """
    Gets the street network graph and nodes and edges GeoDataFrames for a given set of points.

    Parameters:
    gpd (gpd.GeoDataFrame): The input DataFrame containing the point data.

    Returns:
    Tuple[pd.DataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame]: A tuple containing the nodes DataFrame, arcs GeoDataFrame, and convex hull buffer GeoDataFrame.
    """
    start_time = datetime.now()

    logger.info("Getting the bounding box from the DataFrame...")
    # Get the bounding box from the DataFrame
    convex_buffer = get_convex_hull(gdf)

    logger.info("Getting the street network graph from OSM...")
    # Get the street network graph from OSM
    osm_start_time = datetime.now()
    G = ox.graph_from_polygon(
        convex_buffer.geometry.values[0], network_type="drive", simplify=True
    )
    osm_elapsed_time = (datetime.now() - osm_start_time).total_seconds()
    logger.info(f"graph_from_polygon took {osm_elapsed_time:.2f}s to complete.")

    logger.info("Converting the graph to GeoDataFrames...")
    # Convert the graph to GeoDataFrames
    gdf_start_time = datetime.now()
    nodes, arcs = ox.graph_to_gdfs(G, nodes=True, edges=True)
    nodes = nodes.reset_index()
    arcs = arcs.reset_index()
    gdf_elapsed_time = (datetime.now() - gdf_start_time).total_seconds()
    logger.info(f"graph_to_gdfs took {gdf_elapsed_time:.2f}s to complete.")

    # Log the size of the resulting nodes and edges data frames
    num_nodes = len(nodes)
    num_arcs = len(arcs)
    logger.info(f"Nodes GeoDataFrame contains {num_nodes} nodes.")
    logger.info(f"Edges GeoDataFrame contains {num_arcs} edges.")

    end_time = datetime.now()
    logger.info(
        f"get_network function took {(end_time - start_time).total_seconds()} seconds to complete."
    )

    convex_buffer = gpd.GeoDataFrame(
        pd.DataFrame({"__temp": [0]}), geometry=convex_buffer, crs="epsg:4326"
    ).drop(columns=["__temp"])

    return nodes, arcs, convex_buffer

In [138]:
%reload_kedro
lon_col = "parcelLon"
lat_col = "parcelLat"
syn_pop_scenarios = catalog.load("syn_pop_scenarios")
syn_pop = syn_pop_scenarios[list(syn_pop_scenarios.keys())[0]]()
syn_pop = convert_to_geopandas(syn_pop, lat_col=lat_col, lon_col=lon_col)
nodes, arcs, convex_buffer = get_network(syn_pop)

[06/26/23 21:27:31] INFO     Resolved project path as:                                              ]8;id=135799;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=504587;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#136\136]8;;\
                             /Users/ejwillemse/dev/projects/project-rdi-cape-town.                                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/26/23 21:27:32] INFO     Kedro project Project RDI Cape Town                                    ]8;id=788946;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=95748;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#105\105]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=280853;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=306243;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#106\106]8;;\
                             'pipelines'                                                                           

                    INFO     Loading data from 'syn_pop_scenarios' (PartitionedDataSet)...      ]8;id=145954;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=796243;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\

[06/26/23 21:27:34] INFO     Getting the bounding box from the DataFrame...                        ]8;id=536016;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py\4075212042.py]8;;\:]8;id=706349;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py#99\99]8;;\

...

                    INFO     Converting the graph to GeoDataFrames...                             ]8;id=587498;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py\4075212042.py]8;;\:]8;id=304176;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py#112\112]8;;\

[06/26/23 21:27:40] INFO     graph_to_gdfs took 0.68s to complete.                                ]8;id=418352;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py\4075212042.py]8;;\:]8;id=277916;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py#119\119]8;;\

                    INFO     Nodes GeoDataFrame contains 3278 nodes.                              ]8;id=982746;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py\4075212042.py]8;;\:]8;id=562859;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py#124\124]8;;\

                    INFO     Edges GeoDataFrame contains 8507 edges.                              ]8;id=912187;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py\4075212042.py]8;;\:]8;id=381053;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py#125\125]8;;\

                    INFO     get_network function took 5.305289 seconds to complete.              ]8;id=870440;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py\4075212042.py]8;;\:]8;id=605984;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_198/4075212042.py#128\128]8;;\

In [141]:
arcs = arcs.assign(
    **{
        "edge_u": arcs[["u", "v"]].min(axis=1),
        "edge_v": arcs[["u", "v"]].max(axis=1),
        "arc_id": range(len(arcs))
    }
)
arcs = arcs.assign(
    **{
        "edge_u_v_length": arcs["edge_u"].astype(str)
        + "-"
        + arcs["edge_v"].astype(str)
        + "-"
        + arcs["length"].astype(str),
    }
)
candidate_edges = arcs.loc[
    arcs.duplicated(subset=["edge_u_v_length"], keep=False) & (arcs["oneway"] == False)
].sort_values(["edge_u_v_length"])
candidate_edges_count = (
    candidate_edges.groupby(["edge_u_v_length"])
    .agg(**{"count": ("edge_u_v_length", "count")})
    .reset_index()
)
edges_count = candidate_edges_count.loc[candidate_edges_count["count"] == 2]

In [145]:
edges = (
    arcs.loc[arcs["edge_u_v_length"].isin(edges_count["edge_u_v_length"])]
    .copy()[["arc_id", "u", "v", "edge_u_v_length"]]
    .sort_values(["edge_u_v_length"])
)
edges_u = edges.drop_duplicates(subset=["edge_u_v_length"], keep="first")
edges_v = edges.drop_duplicates(subset=["edge_u_v_length"], keep="last")
edges_u = edges_u.assign(inv_edge_id=edges_v["arc_id"].values)
edges_v = edges_v.assign(inv_edge_id=edges_u["arc_id"].values)
edges = pd.concat([edges_u, edges_v]).sort_values(["edge_u_v_length"])
edges

,arc_id,u,v,edge_u_v_length,inv_edge_id
8478,8478,10800412757,10800412766,10800412757-10800412766-43.948,8493
8493,8493,10800412766,10800412757,10800412757-10800412766-43.948,8478
8494,8494,10800412766,10800412758,10800412758-10800412766-58.907,8479
8479,8479,10800412758,10800412766,10800412758-10800412766-58.907,8494
8480,8480,10800412759,10800412764,10800412759-10800412764-11.368,8490
...,...,...,...,...,...
5299,5299,99207133,99207338,99207133-99207338-72.995,5303
7525,7525,1640354274,99207316,99207316-1640354274-35.267,5301
5301,5301,99207316,1640354274,99207316-1640354274-35.267,7525
5300,5300,99207316,99207338,99207316-99207338-72.266,5302


In [ ]:
map_1 = KeplerGl(
    height=1000,
    data={
        "pop": syn_pop.copy(),
        "nodes": nodes.copy(),
        "edges": edges.copy(),
        "boundary": convex_buffer.copy(),
    },
)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


[06/26/23 18:20:34] WARNING  /Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3 ]8;id=329015;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=3386;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             .10/site-packages/jupyter_client/session.py:719: UserWarning: Message                 
                             serialization failed with:                                                            
                             Out of range float values are not JSON compliant                                      
                             Supporting this message is deprecated in jupyter-client 7, please make                
                             sure your message is JSON-compliant                                                   
                               content = self.pack(content)                                                        
                                                                                                                   

In [ ]:
map_1

KeplerGl(data={'pop':             id        income carAccess dwellingTenure housingType  \
0       609273  1.9…

In [ ]:
syn_pop

,id,income,carAccess,dwellingTenure,housingType,mainDwellingType,numberOfDwellingRooms,pipedWater,toilet,parcelLon,parcelLat,parcelX,parcelY,homeToParcelDistance,geometry
0,609273,1.927584e+06,Yes,Rented,House,FormalHouse,7,Piped_dwelling,Flush_sewerage,18.475017,-33.988069,-48507,-3762462,3881,POINT (18.47502 -33.98807)
1,609316,3.011850e+04,No,OwnedPaidOff,House,FormalHouse,4,Piped_dwelling,Flush_sewerage,18.462491,-33.981217,-49669,-3761708,4000,POINT (18.46249 -33.98122)
2,609322,9.637921e+05,Yes,OwnedPaidOff,House,SemiDetachedHouse,4,Piped_dwelling,Flush_sewerage,18.479705,-33.984255,-48076,-3762037,3106,POINT (18.47970 -33.98425)
3,609327,2.409480e+05,No,OwnedPaying,House,FormalHouse,1,Piped_dwelling,Flush_sewerage,18.462500,-33.986328,-49665,-3762275,4220,POINT (18.46250 -33.98633)
4,609371,4.818960e+05,Yes,OwnedPaying,House,FormalHouse,2,Piped_dwelling,Flush_sewerage,18.475533,-33.985806,-48461,-3762211,3595,POINT (18.47553 -33.98581)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38598,1229025,9.637921e+05,Yes,OwnedPaidOff,House,FormalHouse,6,Piped_dwelling,Flush_sewerage,18.478323,-33.983646,-48204,-3761970,4915,POINT (18.47832 -33.98365)
38599,1229028,1.204740e+05,No,Rented,House,Apartment,3,Piped_dwelling,Flush_sewerage,18.481612,-33.987645,-47898,-3762412,4582,POINT (18.48161 -33.98765)
38600,1229073,4.818960e+05,Yes,OwnedPaidOff,House,Apartment,4,Piped_dwelling,Flush_sewerage,18.480235,-33.985628,-48027,-3762189,5404,POINT (18.48024 -33.98563)
38601,1229245,3.855168e+06,Yes,OwnedPaidOff,House,FormalHouse,8,Piped_dwelling,Flush_sewerage,18.480124,-34.020240,-48017,-3766028,2453,POINT (18.48012 -34.02024)


In [68]:
edges = edges.assign(**{"edge_id": range(len(edges))})

In [69]:
syn_pop_xy = syn_pop.to_crs("epsg:3857").drop_duplicates(
    subset=["parcelLon", "parcelLat"]
)

edges_xy = edges.to_crs("epsg:3857")
syn_pop_edge_match = gpd.sjoin_nearest(
    syn_pop_xy, edges_xy, how="left", distance_col="dist__m"
)
syn_pop_edge_match_pure = (
    syn_pop_edge_match.drop_duplicates(subset=["parcelLon", "parcelLat"])
    .drop_duplicates(subset=["id"])
    .reset_index(drop=True)
)
nearest_edges_xy = edges_xy.iloc[syn_pop_edge_match_pure["edge_id"].values].reset_index(drop=True)

pos = nearest_edges_xy.geometry.project(gpd.GeoSeries(syn_pop_edge_match_pure.geometry))
new_pts = nearest_edges_xy.geometry.interpolate(pos)
match_points = new_pts.to_crs("EPSG:4326")
syn_pop_edge_match_pure = syn_pop_edge_match_pure.assign(
    lon_match=match_points.x.values, lat_match=match_points.y.values
)[["parcelLon", "parcelLat", "lon_match", "lat_match", "edge_id", "dist__m"]]

syn_pop = syn_pop.drop(columns=["geometry"]).merge(syn_pop_edge_match_pure).merge(
    edges, left_on="edge_id", right_on="edge_id", validate="m:1"
)

In [ ]:
map_2 = KeplerGl(
    height=1000,
    data={
        "pop": syn_pop.copy(),
        "nodes": nodes.copy(),
        "edges": edges.copy(),
        "match": syn_pop_edge_match_pure.copy(),
    },
)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


[06/26/23 20:08:33] WARNING  /Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3 ]8;id=605381;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=239518;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             .10/site-packages/jupyter_client/session.py:719: UserWarning: Message                 
                             serialization failed with:                                                            
                             Out of range float values are not JSON compliant                                      
                             Supporting this message is deprecated in jupyter-client 7, please make                
                             sure your message is JSON-compliant                                                   
                               content = self.pack(content)                                                        
                                                                                                                   

In [ ]:
map_2

KeplerGl(data={'pop':             id        income carAccess dwellingTenure housingType  \
0       609273  1.9…